In [419]:
import kfp
import kfp.components as comp
import requests
import kfp.dsl as dsl

In [420]:
def preparation_data_for_clusturing():
    import numpy as np
    import pandas as pd
    insurance = pd.read_csv('data/unsupervised-lerning-data-insurance.csv', sep=';')
    columns_to_drops = ['ID', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence','Distribution_channel', 'Policies_in_force','Max_policies', 'Max_products','Date_lapse', 'Lapse', 'Payment', 'Premium', 'Cost_claims_year',
       'N_claims_year', 'N_claims_history', 'R_Claims_history', 'Area', 'Second_driver', 'Year_matriculation', 'Length'  ,'Type_risk' ]
    insurance = insurance.drop(columns= columns_to_drops, axis=1)
    insurance.drop_duplicates(inplace=True)
    insurance = insurance.dropna()
    insurance_type = {
    'P':1,
    'D':2
    }
    insurance['Type_fuel'] = insurance['Type_fuel'].map(insurance_type)
    insurance.to_csv('data/preparation_data_for_clusturing.csv', index=False)
    

In [421]:
def clusturing_and_save_supervised_data():
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    import numpy as np
    import pandas as pd
    k_value =  3
    insurance = pd.read_csv('data/preparation_data_for_clusturing.csv')
    datascaler = StandardScaler()
    data_numeric= insurance[['Seniority','Power', 'Cylinder_capacity', 'Value_vehicle', 'N_doors', 'Weight']]
    data_insurance_scaled = datascaler.fit_transform(data_numeric)
    kmeans = KMeans(n_clusters=k_value, init='k-means++', max_iter=300, n_init=10, random_state=0)
    insurance['Cluster'] = kmeans.fit_predict(data_insurance_scaled)
    insurance.to_csv('data/unbalanced_supervised-lerning-data-insurance.csv', index=False)

In [422]:
def unbalanced_data_procissing_to_balanced():
    import numpy as np
    import pandas as pd
    from sklearn.utils import resample
    insurance = pd.read_csv('data/unbalanced_supervised-lerning-data-insurance.csv')
    print(insurance['Cluster'].value_counts())
    min_observation_count = insurance['Cluster'].value_counts().min()
    
    balanced_dfs = []

    for cluster_label, group in insurance.groupby('Cluster'):
        
        balanced_cluster = resample(group, replace=False, n_samples=min_observation_count, random_state=42)
    
        balanced_dfs.append(balanced_cluster)

    balanced_insurance = pd.concat(balanced_dfs, ignore_index=True)

    print(balanced_insurance['Cluster'].value_counts())
    print(balanced_insurance.shape)
    print(balanced_insurance.info())
    balanced_insurance.to_csv('data/balanced_supervised-lerning-data-insurance.csv', index=False)

In [423]:
def train_test_split():
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import MinMaxScaler
    import numpy as np
    import pandas as pd
    import pickle
    balanced_insurance = pd.read_csv('data/balanced_supervised-lerning-data-insurance.csv')
    X = balanced_insurance.drop('Cluster', axis=1)  # Exclude the 'Cluster' column
    y = balanced_insurance['Cluster']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    np.save(f'data/y_train.npy', y_train)
    np.save(f'data/y_test.npy', y_test)
    np.save(f'data/X_train.npy', X_train)
    np.save(f'data/X_test.npy', X_test)
    with open('data/minmaxscaler.pkl', 'wb') as scaler_file:
        pickle.dump(scaler, scaler_file)
        
    print("\n---- X_train ----")
    print("\n")
    print(X_train)
    
    print("\n---- X_test ----")
    print("\n")
    print(X_test)
    
    print("\n---- y_train ----")
    print("\n")
    print(y_train)
    
    print("\n---- y_test ----")
    print("\n")
    print(y_test)

In [424]:
def evaluate_models_level0():
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.tree import ExtraTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import BaggingClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import AdaBoostClassifier
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import confusion_matrix
    import numpy as np
    import pandas as pd
    import pickle
    models = {
        'Logistic Regression': LogisticRegression(),
        'Naive Bayes': GaussianNB(),
        'Support Vector Machine': SVC(),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Bagging': BaggingClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'Extra Trees': ExtraTreeClassifier(),
    }

    y_train = np.load(f'data/y_train.npy', allow_pickle=True)
    y_test = np.load(f'data/y_test.npy', allow_pickle=True)
    X_train = np.load(f'data/X_train.npy', allow_pickle=True)
    X_test = np.load(f'data/X_test.npy', allow_pickle=True)

    with open(f'data/minmaxscaler.pkl', 'rb') as f:
        scaler = pickle.load(f)

    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    print(X_train)
    for name, model in models.items():
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
            recall = recall_score(y_test, y_pred, average='weighted')
            f1 = f1_score(y_test, y_pred, average='weighted')

            confusion_mat = confusion_matrix(y_test, y_pred)

            print(f"{name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
            print(f"Confusion Matrix:\n{confusion_mat}\n{'-'*50}")
        except Exception as e:
            print(f"Error occurred for {name}: {e}")


In [425]:
def train_model_staking():
    from sklearn.linear_model import LogisticRegression
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import AdaBoostClassifier
    import numpy as np
    import pandas as pd
    import pickle
    from sklearn.ensemble import StackingClassifier

    y_train = np.load(f'data/y_train.npy',allow_pickle=True)
    y_test = np.load(f'data/y_test.npy', allow_pickle=True)
    X_train = np.load(f'data/X_train.npy', allow_pickle=True)
    X_test = np.load(f'data/X_test.npy', allow_pickle=True)
    with open(f'data/minmaxscaler.pkl','rb') as f:
        scaler = pickle.load(f)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)



    level0 = [
    ('AdaBoost', AdaBoostClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('Support Vector Machine', SVC()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Random Forest', RandomForestClassifier())
    ]
    
    # Define level1 classifier
    level1 = LogisticRegression(max_iter=1000)
    
    # Create StackingClassifier
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    
    # Fit the model
    model.fit(X_train, y_train)
    

    with open('data/model.pkl', 'wb') as model_file:
        pickle.dump(model, model_file)
        
    # Predictions
    y_pred = model.predict(X_test)
    np.save(f'data/y_pred.npy', y_pred) 
    print(y_pred)
    

In [426]:
def evaluate_model_staking():
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
    from sklearn.metrics import confusion_matrix
    import numpy as np
    import pandas as pd
    import pickle
    
    with open(f'data/model.pkl','rb') as f:
        model = pickle.load(f)
    y_test = np.load(f'data/y_test.npy', allow_pickle=True) 
    X_test = np.load(f'data/X_test.npy', allow_pickle=True)
    y_pred = np.load(f'data/y_pred.npy', allow_pickle=True)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    #matrix confision
    matrix = confusion_matrix(y_test, y_pred)

    print(f"Stacking Classifier - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
    print(matrix)
    

# Kubeflow pipeline creation 

In [427]:
create_step_preparation_data = kfp.components.create_component_from_func(
    func=preparation_data_for_clusturing,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2']
)

In [428]:
create_step_clusturing_and_save_supervised_data = kfp.components.create_component_from_func(
    func=clusturing_and_save_supervised_data,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [429]:
create_step_unbalanced_data_procissing_to_balanced = kfp.components.create_component_from_func(
    func=unbalanced_data_procissing_to_balanced,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [430]:
create_step_train_test_split = kfp.components.create_component_from_func(
    func=train_test_split,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [431]:
create_step_evaluate_models_level0 = kfp.components.create_component_from_func(
    func=evaluate_models_level0,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [432]:
create_step_train_model_staking = kfp.components.create_component_from_func(
    func=train_model_staking,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [433]:
create_step_evaluate_model_staking = kfp.components.create_component_from_func(
    func=evaluate_model_staking,
    base_image='python:3.9',
    packages_to_install=['pandas == 2.1.3','numpy == 1.26.2','scikit-learn==1.3.2']
)

In [434]:
# Define the pipeline
@dsl.pipeline(
   name='Insurance Recomendation deploy in kubflow',
   description='ML pipeline  de System recomendation'
)
# Define parameters to be fed into pipeline
def systeme_recomendation_pipeline(data_path: str):
    vop = dsl.VolumeOp(
    name="systme-recomendation_-vol",
    resource_name="systme-recomendation_vol", 
    size="2Gi", 
    modes=dsl.VOLUME_MODE_RWO)
    
    preparation_data_for_clustring = create_step_preparation_data().add_pvolumes({data_path: vop.volume})
    clusturing_and_save_supervised_data = create_step_clusturing_and_save_supervised_data().add_pvolumes({data_path: vop.volume}).after(preparation_data_for_clustring)
    unbalanced_data_procissing_to_balanced = create_step_unbalanced_data_procissing_to_balanced().add_pvolumes({data_path: vop.volume}).after(clusturing_and_save_supervised_data)
    train_test_split = create_step_train_test_split().add_pvolumes({data_path: vop.volume}).after(unbalanced_data_procissing_to_balanced)
    evaluate_models_level0 = create_step_evaluate_models_level0().add_pvolumes({data_path: vop.volume}).after(train_test_split)
    train_model_staking = create_step_train_model_staking().add_pvolumes({data_path: vop.volume}).after(evaluate_models_level0)
    evaluate_model_staking = create_step_evaluate_model_staking().add_pvolumes({data_path: vop.volume}).after(train_model_staking)
    
    preparation_data_for_clustring.execution_options.caching_strategy.max_cache_staleness = "P0D"
    clusturing_and_save_supervised_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    unbalanced_data_procissing_to_balanced.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train_test_split.execution_options.caching_strategy.max_cache_staleness = "P0D"
    evaluate_models_level0.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train_model_staking.execution_options.caching_strategy.max_cache_staleness = "P0D"
    evaluate_model_staking.execution_options.caching_strategy.max_cache_staleness = "P0D"

In [435]:
kfp.compiler.Compiler().compile(
    pipeline_func=systeme_recomendation_pipeline,
    package_path='system_recomendation_insurance_pipeline1.yaml')